# 4_Model_Testing

En este notebook trataremos de poner a prueba nuestros modelos entrenados con nuevos datos. Desde que se genero el dataset hasta la fecha actual 07-Sep, Han salido nuevos hits que veremos que modelo de los anteriores es capaz de predecir mas hits.

In [69]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '../Data_Generation')
from music_data import Music_Data
import joblib
import lightgbm as lgb

In [3]:
df_20 = pd.read_csv("../Data/top_100_Billboard_2020.csv")

In [5]:
df_20['date_chart'].max()

'2020-08-08'

In [10]:
data_music = Music_Data("01ec25b32db847e3aeb622ebe1a894c8", "cd8d3b7666164ea5943688215d9226c2", "98Uam09gIfBOQczz40AbY0uOxxjF4tOtnu9IptsLvtpyEgnkQzgqnBq0Roo9jQND")

In [12]:
df_test = data_music.Get_Billboard_Data("09-08-2020","07-09-2020")

Added song: Watermelon Sugar Artist: Harry Styles
Added song: Rockstar Artist: DaBaby 
Added song: Whats Poppin Artist: Jack Harlow 
Added song: Blinding Lights Artist: The Weeknd
Added song: Roses Artist: SAINt JHN
Added song: My Future Artist: Billie Eilish
Added song: Savage Love (Laxed - Siren Beat) Artist: Jawsh 685 x Jason Derulo
Added song: Cardigan Artist: Taylor Swift
Added song: Go Crazy Artist: Chris Brown & Young Thug
Added song: Blueberry Faygo Artist: Lil Mosey
Added song: Before You Go Artist: Lewis Capaldi
Added song: Adore You Artist: Harry Styles
Added song: Savage Artist: Megan Thee Stallion 
Added song: Break My Heart Artist: Dua Lipa
Added song: I Hope Artist: Gabby Barrett
Added song: Circles Artist: Post Malone
Added song: Come & Go Artist: Juice WRLD x Marshmello
Added song: Popstar Artist: DJ Khaled 
Added song: Move Ya Hips Artist: A$AP Ferg 
Added song: For The Night Artist: Pop Smoke 
Added song: Wishing Well Artist: Juice WRLD
Added song: Don't Start Now Ar

In [26]:
df_20.drop(df_20.columns[3:30], axis = 1, inplace = True) 
df_test.drop(df_test.columns[3:30], axis = 1, inplace = True) 

In [27]:
df_uniq = df_20.merge(df_test, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

In [30]:
df_uniq = df_uniq.drop_duplicates(subset = ["id"])
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es la misma canción
# por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la canción, para solventar esta
# casuistica.
df_uniq = df_uniq.drop_duplicates(subset = ["artist","title"], keep="first")

In [47]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='01ec25b32db847e3aeb622ebe1a894c8', client_secret='cd8d3b7666164ea5943688215d9226c2') 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=50)

def find_song(id_track):
    try:
        track = sp.track(id_track)
        track_feat = sp.audio_features(id_track)
        song = [[track_feat[0]["danceability"], track_feat[0]["energy"], track_feat[0]["key"],
        track_feat[0]["loudness"], track_feat[0]["mode"], track_feat[0]["speechiness"],
        track_feat[0]["acousticness"], track_feat[0]["instrumentalness"], track_feat[0]["liveness"],
        track_feat[0]["valence"], track_feat[0]["tempo"],track_feat[0]["time_signature"],
                track["duration_ms"]]]
        
        return song
    except:
        print ("ERROR: Titulo no encontrado")

In [48]:
filename = 'light_gbm_model_random.pkl'
best_lgb = joblib.load(filename) 

In [66]:
df_uniq['proba_lgb']= 0 
for i, item in df_uniq.iterrows():
    song = find_song(item['id'])
    score = best_lgb.predict_proba(song)
    df_uniq.loc[i,"proba_lgb"]=score[0][1]

In [68]:
len (df_uniq)

34

In [67]:
len (df_uniq)-len(df_uniq[df_uniq['proba_lgb']<0.5])

15